In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
import itertools
from datasets import load_dataset
from sentence_transformers import SentenceTransformer , SentenceTransformerTrainer
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss ,ContrastiveLoss, OnlineContrastiveLoss
import os
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
import torch

# Imports
import pandas as pd
from datasets import load_dataset , Dataset
from transformers import RobertaTokenizer , RobertaForSequenceClassification , TrainingArguments , Trainer,TrainerCallback,AutoModelForSequenceClassification,AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support , accuracy_score
import torch
import numpy as np
import torch.nn as nn
import numpy as np
from datasets import Dataset, Features, Value,Sequence
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score

/home/u111169/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
Could not load bitsandbytes native library: /share/apps/eb/Anaconda3/2023.03-1/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /home/u111169/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/home/u111169/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
  File "/home/u111169/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
  File "/share/apps/eb/Anaconda3/2023.03-1/lib/python3.10/ctypes/__init__.py", line 452, in LoadLibrary
    return self._dlltype(name)
  File "/share/apps/eb/Anaconda3/2023.03-1/lib/python3.10/ctypes

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
import random

In [6]:
def get_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES")

    # Generate Morgan fingerprint (radius=2, 2048-bit vector)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)

    # Convert to a bit string or list
    bit_string = fp.ToBitString()
    # bit_list = list(fp)
    return bit_string

In [7]:
def get_all_non_canonicals(original_smiles): 
    mol = Chem.MolFromSmiles(original_smiles)
    if mol is None:
        print("Invalid SMILES")
    else:
        num_atoms = mol.GetNumAtoms()
        seen = set()
        # Generate multiple non-canonical SMILES
        for _ in range(1000):  # Adjust iterations for more coverage
            new_order = list(range(num_atoms))
            random.shuffle(new_order)
            randomized_mol = Chem.RenumberAtoms(mol, new_order)
            randomized_smiles = Chem.MolToSmiles(randomized_mol, canonical=False)
            if randomized_smiles not in seen:
                seen.add(randomized_smiles)
    return list(seen)

In [25]:
def df2noncanonical(df): 
    f_l = []
    for n , i in df["SMILES"].items(): 
        df_c = pd.DataFrame()
        df_c["SMILES"] = get_all_non_canonicals(i)
        df_c["labels"] = df["labels"][n]
        f_l.append(df_c)
    df_train_with_noncanonicals = pd.concat(f_l , axis=0)
    
    return df_train_with_noncanonicals

In [2]:
print("Imports completed")

os.environ["WANDB_DISABLED"] = "true"

df0 = pd.read_csv("/home/u111169/wrkdir/mgh-project/ChemAP/dataset/DrugApp/All_training_feature_vectors.csv")
df=df0[["SMILES",	"Label"]]

print("dataset created ")

Imports completed
dataset created 


In [3]:
from sklearn.model_selection import train_test_split

df.rename(columns={"Label":"labels"}, inplace=True)

test_size = 0.2
val_size = 0.5
train_df , temp = train_test_split(df , stratify = df.labels , test_size = test_size , random_state=1234)
test_df , val_df = train_test_split(temp , stratify = temp.labels , test_size = val_size , random_state=1234)
#-----------------------------------
# reset index
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_42908/3489636595.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Label":"labels"}, inplace=True)


In [13]:
df_train_with_noncanonicals

,0
0,OC([C@@H]1[C@@H](O)[C@H](O)[C@@H](O[C@H]2[C@H]...
1,[C@@H]12C[C@](C(=O)O)(C)CC[C@]1(C)CC[C@]1(C)C2...
2,O1[C@H](C(=O)O)[C@@H](O)[C@H](O)[C@@H](O)[C@@H...
3,[C@]12(C)CC[C@H](O[C@@H]3[C@H](O[C@H]4[C@H](O)...
4,C[C@]12[C@@]3(C)C(=CC(=O)[C@@H]1[C@]1(C)[C@H](...
...,...
47,[C@H]12[C@H](OC[C@H]1O)[C@@H](O)CO2
48,C1[C@H](O)[C@H]2OC[C@@H](O)[C@H]2O1
49,O1[C@@H]2[C@H](O)CO[C@@H]2[C@@H](O)C1
50,[C@@H]1(O)CO[C@@H]2[C@@H](O)CO[C@H]12


In [ ]:
#-----------------------------------
# convert the dataframes to huggingface dataset for easier upload on hub and eaiser accessibility
dataset_train = Dataset.from_pandas(train_df)
dataset_df = Dataset.from_pandas(val_df)
dataset_test = Dataset.from_pandas(test_df)
#tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")#'FacebookAI/xlm-roberta-large'
tokenizer = RobertaTokenizer.from_pretrained("/home/u111169/.cache/huggingface/hub/models--DeepChem--ChemBERTa-77M-MTR/snapshots/66b895cab8adebea0cb59a8effa66b2020f204ca")#'FacebookAI/xlm-roberta-large'


def tokenize_function(examples):
    return tokenizer(examples['SMILES'], padding="max_length" , truncation=True )#,max_length=166

tokenized_train = dataset_train.map(tokenize_function)
tokenized_val = dataset_df.map(tokenize_function)
tokenized_test = dataset_test.map(tokenize_function)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor([len(train_df[train_df["labels"] == i]) / len(train_df) for i in np.unique(train_df["labels"])]).to(device)
#class_weights = torch.tensor([0.25,0.25,0.25,0.25]).to(device)

class CustomLoss(nn.Module):
    def __init__(self, class_weights):
        super(CustomLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, logits, labels):
        ce_loss = self.ce_loss(logits, labels)
        l1_loss = torch.mean(torch.abs(logits))
        return ce_loss + 0.01 * l1_loss

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = CustomLoss(class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
    


model = RobertaForSequenceClassification.from_pretrained("/home/u111169/.cache/huggingface/hub/models--DeepChem--ChemBERTa-77M-MTR/snapshots/66b895cab8adebea0cb59a8effa66b2020f204ca" ,num_labels=2)
model.config.classifier_dropout=0.01

os.environ["WANDB_DISABLED"] = "true"



from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir='/home/u111169/wrkdir/mgh-project/arxive/checkpoints-arxive/fine_tuning_chemberta_classic_way/trainer-not-custom',
    num_train_epochs= 15,
    evaluation_strategy='steps',
    save_strategy='steps',
    learning_rate=2e-3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_steps=20,
    #save_total_limit=3,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    do_eval=True,
    do_train=True,
    weight_decay=0.1,
    #logging_dir = "logs",
    logging_strategy="steps",
    logging_steps = 10,
    dataloader_drop_last=True,
    save_safetensors=False,
    adam_epsilon=1e-08,
    warmup_steps=100,
    seed=42,
    lr_scheduler_type='cosine',
    load_best_model_at_end = True,
    label_smoothing_factor=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer = tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]

)
trainer.train()